In [ ]:
#Zip dosyasında bulunan veri setini okuma
import zipfile
import os

zip_file = "/content/drive/MyDrive/retina_optic_disc.zip"
new_file_name = "retina_optik"

os.makedirs(new_file_name,exist_ok=True)

with zipfile.ZipFile(zip_file,"r") as zip_fol:
  zip_fol.extractall(new_file_name)

In [ ]:
#Gerekli Kütüphanelerin Yüklenmesi
import cv2
import numpy as np
import seaborn as sns
import random
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from PIL import Image

In [ ]:
#Verilerin bulunduğu dosya yolunun verilmesi
data_dir = "//content//retina_optik//retina_optic_disc"

In [ ]:
import os
train_dir = os.path.join(data_dir  + "/train")
print("Train Directory --> ", os.listdir(train_dir))

validation_dir = os.path.join(data_dir  + "/val")
print("Validation Directory --> ", os.listdir(validation_dir))

test_dir = os.path.join(data_dir  + "/test")
print("Test Directory --> ", os.listdir(test_dir))

In [ ]:
import pandas as pd


def create_dataframe(data_path):
    filepaths = []
    labels = []

    # Ana klasördeki alt klasörleri  listeleme
    folds = os.listdir(data_path)
    for fold in folds:
        # Alt klasördeki görüntülerin yollarını almak için yol oluşturma
        f_path = os.path.join(data_path, fold)
        imgs = os.listdir(f_path)  # Alt klasördeki görüntüleri listeleme

        # Her görüntü için dosya yolunu ve sınıf etiketini ekleme
        for img in imgs:
            img_path = os.path.join(f_path, img)  # Görselin tam dosya yolunu oluşturma
            filepaths.append(img_path)  # Görselin yolunu listeye ekleme
            labels.append(fold)  # Görselin etiketini listeye ekleme

    
    fseries = pd.Series(filepaths, name='Filepaths')
    lseries = pd.Series(labels, name='Labels')

    # İki seriyi birleştirip bir DataFrame olarak döndürme
    return pd.concat([fseries, lseries], axis=1)


train_retina_df = create_dataframe(train_dir)
test_retina_df = create_dataframe(test_dir)
validation_retina_df = create_dataframe(validation_dir)

print(train_retina_df)
print(test_retina_df)
print(validation_retina_df)

In [ ]:
final_train_df=pd.concat([train_retina_df],ignore_index=True)
print(final_train_df)
final_test_df = pd.concat([test_retina_df],ignore_index=True)
print(final_test_df)
final_validation_df=pd.concat([validation_retina_df],ignore_index=True)
print(final_validation_df)

In [ ]:
import tensorflow as tf
from tensorflow.python.keras.models import Sequential
from keras.src.legacy.preprocessing.image import ImageDataGenerator
from keras.src.layers import Conv2D, MaxPooling2D, Flatten, Dense, Activation, Dropout, BatchNormalization
from keras.src.utils import image_dataset_from_directory

# ResNet50 modelini oluşturur ve yapılandırır.
resnet_50 = tf.keras.applications.ResNet50(
    include_top=False,       
    weights='imagenet',      
    input_tensor=None,      
    input_shape=(224, 224, 3),  
    pooling=None,           
    classes=1000            
)

train_datagen = ImageDataGenerator(
    rescale=1./255,              
    rotation_range=20,           
    width_shift_range=0.2,     
    height_shift_range=0.2,      
    shear_range=0.2,             
    zoom_range=0.2,              
    horizontal_flip=True,        
    fill_mode='nearest'
)


valid_datagen = ImageDataGenerator(rescale=1./255) 
test_datagen = ImageDataGenerator(rescale=1./255)   


train_generator = train_datagen.flow_from_dataframe(
    dataframe=final_train_df,           
    x_col='Filepaths',            
    y_col='Labels',               
    class_mode='categorical',     
    shuffle=True,                 
    batch_size=32,                
    seed=42                       
)


valid_generator = valid_datagen.flow_from_dataframe(
    dataframe=final_validation_df,           
    x_col='Filepaths',            
    y_col='Labels',               
    class_mode='categorical',    
    shuffle=False,               
    batch_size=32                
)


test_generator = test_datagen.flow_from_dataframe(
    dataframe=final_test_df,            
    x_col='Filepaths',            
    y_col='Labels',               
    class_mode='categorical',     
    shuffle=False,                
    batch_size=32                 
)

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam

# ResNet50 modelini oluşturur ve yapılandırır
base_model = tf.keras.applications.ResNet50V2(
    weights='imagenet',          
    include_top=False,           
    input_shape=(224, 224, 3)    
)

# Modelin katmanlarını oluşturur
model = Sequential([
    base_model,                  # ResNet50 modelini ekler
    GlobalAveragePooling2D(),    
    Dense(512, activation='relu'),  
    Dropout(0.3),                
    Dense(4, activation='softmax')  
])

# Modelin derlenmesi
model.compile(
    optimizer=Adam(learning_rate=0.0001),  
    loss='categorical_crossentropy',        # Çok sınıflı sınıflandırma için uygun olan categorical_crossentropy kaybı kullanılır
    metrics=['accuracy']                    
)

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    horizontal_flip=True,
    fill_mode='nearest'
)


valid_datagen = ImageDataGenerator(rescale=1./255)


train_generator = train_datagen.flow_from_dataframe(
    dataframe=final_train_df,
    x_col='Filepaths',
    y_col='Labels',
    target_size=(224, 224),
    class_mode='categorical',
    shuffle=True,
    batch_size=32,
    seed=42
)


valid_generator = valid_datagen.flow_from_dataframe(
    dataframe=final_validation_df,
    x_col='Filepaths',
    y_col='Labels',
    target_size=(224, 224),
    class_mode='categorical',
    shuffle=False,
    batch_size=32
)

# Modelin eğitilmesi
history = model.fit(
    x=train_generator,                
    validation_data=valid_generator,  
    epochs=25,                        
    verbose=1,                                                                    
    shuffle=False,                    
    validation_steps=None             
)



In [ ]:
# Modelin test klasöründeki veriler üzerinde değerlendirilmesi

test_loss, test_accuracy = model.evaluate(test_generator)
print('Test Accuracy:',test_accuracy)


In [ ]:
# modelin eğitim sürecini ve doğrulama sürecini görselleştiren kod parçası
import matplotlib.pyplot as plt

def plot_training_history(history):
   
    tr_acc = history.history['accuracy']
    tr_loss = history.history['loss']
    val_acc = history.history['val_accuracy']
    val_loss = history.history['val_loss']
    epochs = [i + 1 for i in range(len(tr_acc))]

    plt.figure(figsize=(20, 8))

    
    plt.subplot(1, 2, 1)
    plt.plot(epochs, tr_loss, 'r', label='Eğitim Kaybı')
    plt.plot(epochs, val_loss, 'g', label='Doğrulama Kaybı')
    plt.title('Eğitim ve Doğrulama Kaybı')
    plt.xlabel('Epoch')
    plt.ylabel('Kayıp')
    plt.legend()

    
    plt.subplot(1, 2, 2)
    plt.plot(epochs, tr_acc, 'r', label='Eğitim Doğruluğu')
    plt.plot(epochs, val_acc, 'g', label='Doğrulama Doğruluğu')
    plt.title('Eğitim ve Doğrulama Doğruluğu')
    plt.xlabel('Epoch')
    plt.ylabel('Doğruluk')
    plt.legend()

    plt.tight_layout()
    plt.show()

# Eğitim sürecini görselleştirme fonksiyonunu çağırıyoruz
plot_training_history(history)


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report

test_datagen = ImageDataGenerator(rescale=1./255)


test_generator = test_datagen.flow_from_directory(
    '/content/retina_optik/retina_optic_disc/test',  # Test veri seti klasörünün yolu
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

# Girdi veri şeklini kontrol etme
for data_batch, labels_batch in test_generator:
    print(f"Data batch shape: {data_batch.shape}")
    print(f"Labels batch shape: {labels_batch.shape}")
    break  # Bir batch'i kontrol ettikten sonra döngüden çık


# Test verileriyle tahmin yap
y_pred = model.predict(test_generator)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = test_generator.classes

# Karmaşıklık matrisi
conf_matrix = confusion_matrix(y_true, y_pred_classes)
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues',
            xticklabels=test_generator.class_indices.keys(),
            yticklabels=test_generator.class_indices.keys())
plt.title('Confusion Matrix')
plt.xlabel('Tahmin Edilen Etiketler')
plt.ylabel('Gerçek Etiketler')
plt.show()

# Sınıflandırma raporu
class_report = classification_report(y_true, y_pred_classes, target_names=test_generator.class_indices.keys())
print("Sınıflandırma Raporu:")
print(class_report)


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os

# Random bir örnek seçme ve görseli gösterme
random_sample = final_test_df.sample(n=1).iloc[0]
sample_path = random_sample['Filepaths']
sample_label = random_sample['Labels']
sample_img = plt.imread(sample_path)

# Görseli gösterme
plt.imshow(sample_img, cmap='gray')
plt.text(10, 10, f'Görsel ismi: {os.path.basename(sample_path)}', color='white', fontsize=12, bbox=dict(facecolor='black', alpha=0.5))
plt.axis('off')
plt.show()

if sample_img is not None:
    # Görseli yeniden boyutlandırma ve boyutları kontrol etme
    sample_img_resized = cv2.resize(sample_img, (224, 224))  # ResNet50V2 modeline uygun boyut
    sample_img_normalized = sample_img_resized / 255.0  # Normalizasyon
    
    # Gri tonlamalı görüntüyü 3 kanallı hale getirme
    if len(sample_img_normalized.shape) == 2:  # Görüntü gri tonlamalıysa
        sample_img_normalized = np.stack((sample_img_normalized,)*3, axis=-1)
        
    sample_img_expanded = np.expand_dims(sample_img_normalized, axis=0)  # Batch boyutunu ekleme

    # Boyutları kontrol etme
    print(f"Resized image shape: {sample_img_resized.shape}")
    print(f"Expanded image shape: {sample_img_expanded.shape}")

    # Model Tahmini
    prediction = model.predict(sample_img_expanded)  # Burada batch boyutunu kullanıyoruz
    class_names = ['CNV', 'DME', 'DRUSEN', 'HEALTY']
    predicted_class = np.argmax(prediction)
    predicted_label = class_names[predicted_class]

    # Tahmin sonucunu gösterme
    plt.imshow(sample_img_resized, cmap='gray')
    plt.title(f'Tahmin Sonucu: {predicted_label}')
    plt.axis('off')
    plt.show()
else:
    print("Görsel okunamadı.")
